# Deep Belief Networks

In [43]:
import tensorflow as tf
import keras
# tf.config.run_functions_eagerly(True)

from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam

In [44]:
# Load the dataset 
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() 

In [45]:
# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten the data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [46]:
# Build the DBN
input_layer = Input(shape=(784,))
hidden_layer = Dense(512, activation='relu')(input_layer)
output_layer = Dense(10, activation='softmax')(hidden_layer)

model = Model(input_layer, output_layer)

In [47]:
# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'], jit_compile=True)

In [48]:
# Train the model
with tf.device('/CPU:0'):    
    model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.2027 - accuracy: 0.9395 - val_loss: 0.0999 - val_accuracy: 0.9704
Epoch 2/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0816 - accuracy: 0.9746 - val_loss: 0.0706 - val_accuracy: 0.9785
Epoch 3/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0531 - accuracy: 0.9828 - val_loss: 0.0654 - val_accuracy: 0.9797
Epoch 4/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0359 - accuracy: 0.9886 - val_loss: 0.0737 - val_accuracy: 0.9778
Epoch 5/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.0671 - val_accuracy: 0.9805
Epoch 6/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0213 - accuracy: 0.9931 - val_loss: 0.0712 - val_accuracy: 0.9793
Epoch 7/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0171 - accuracy: 0.9945 - val_loss: 0.0747 -

## a.

In [49]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 3s 11ms/step - loss: 0.0804 - accuracy: 0.9817
Test accuracy: 0.9817000031471252


## b.

## c.